In [1]:
import requests

In [7]:
import requests
import html2text

from bs4 import BeautifulSoup
from html2text import HTML2Text
from abc import ABC, abstractmethod


class Parser(ABC):
    """
    Interface for Parsing Strings
    """

    @abstractmethod
    def parse(self, text: str) -> str:
        return text

class HTMLParser(Parser):
    def __init__(self, html2text: HTML2Text, ids: list[str] = [], tags: list[str] = []):
        self.h = html2text
        self.ids = ids
        self.tags = tags

    def parse(self, text: str) -> str:
        soup = BeautifulSoup(text, "html.parser")

        # First, look for elements by id in provided order
        for element_id in self.ids:
            element = soup.find(id=element_id)
            if element:
                return self.h.handle(str(element))

        # Next, look for the first available element by tag name
        for tag in self.tags:
            element = soup.find(tag)
            if element:
                return self.h.handle(str(element))

        # Fallback: process the entire HTML
        return self.h.handle(text)


def fetch_and_convert(url):
    """
    Fetches the content from the given URL and converts it to readable text using html2text.
    """
    response = requests.get(url)
    response.raise_for_status()  # Raises an error if the request failed

    html_content = response.body

    # Initialize html2text converter
    h = html2text.HTML2Text()
    h.ignore_links = False        # Set to True to skip link formatting
    h.body_width = 0              # Disable line wrapping for raw readability

    text = h.handle(html_content)
    return HTMLParser(h, ids=["content"]).parse(html_content)

if __name__ == "__main__":
    url = "https://www.hawaii.edu/policy/index.php?action=viewPolicy&policySection=ep&policyChapter=5&policyNumber=215"
    plain_text = fetch_and_convert(url)
    print(plain_text)


AttributeError: 'Response' object has no attribute 'body'

In [11]:
import requests
from readability import Document

url = "https://www.hawaii.edu/policy/index.php?action=viewPolicy&policySection=ep&policyChapter=5&policyNumber=215"

html = requests.get(url).text

doc = Document(html)
main_content_html = doc.summary()      # HTML of the main article
main_content_text = doc.title() + "\n" + Document(html).summary()

print(main_content_text)

ImportError: cannot import name 'Document' from 'readability' (/Users/rodericktabalba/miniforge3/envs/house_finance_env/lib/python3.11/site-packages/readability/__init__.py)